In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [39]:
# Set up plotting style
sns.set_style("whitegrid")
# Set font for better compatibility (adjust if specific font is needed)
plt.rcParams['figure.figsize'] = (15, 10) 

In [41]:
# 0. Configuration
# ------------------------------------
processed_dir = "/Users/wanderer/DSS_Thesis/data/processed"
file_name = "n_indresp_cleaned_02.csv"
file_path = os.path.join(processed_dir, file_name)

In [43]:
# --- GHQ Items List and UKHLS Missing Codes ---
UKHLS_MISSING_CODES = [-9.0, -8.0, -7.0, -2.0, -1.0]

# --- Helper Function 1: Clean Missing Codes (for float conversion) ---
def clean_missing_codes(df, cols):
    """Replaces UKHLS special negative codes with NaN and ensures float type."""
    # Note: Use errors='ignore' in astype(float) if the column might contain non-numeric strings
    for col in cols:
        if col in df.columns:
            df[col] = df[col].replace(UKHLS_MISSING_CODES, np.nan).astype(float, errors='ignore')
    return df

# --- Helper Function 2: Clean and Convert to Nullable Integer ---
def clean_and_convert_to_nullable_int(df, cols):
    """Converts UKHLS missing codes to NaN and converts to Pandas' nullable Int64."""
    for col in cols:
        if col in df.columns:
            df[col] = df[col].replace(UKHLS_MISSING_CODES, np.nan)
            try:
                # Convert to nullable Int64 (supports NaN)
                df[col] = df[col].astype('Int64')
            except Exception:
                # Fallback to float if conversion to Int64 fails
                df[col] = df[col].astype(float)
    return df

def clean_and_convert_to_nullable_float(df, columns):
    """
    Replace UKHLS-style special missing codes (-9, -8, -7, -2, -1) with np.nan
    and convert columns to nullable float dtype.
    """
    for col in columns:
        if col in df.columns:
            df[col] = df[col].replace([-9, -8, -7, -2, -1], np.nan)
            df[col] = df[col].astype(float)
    return df


# ==============================================================================
# 0. Load Data and Initial Cleaning
# ==============================================================================
try:
    df = pd.read_csv(file_path) 
    print(f"Successfully loaded data from: {file_path}")
except FileNotFoundError:
    print(f"Error: File not found at {file_path}. Please check the path and filename.")
    exit() 
except Exception as e:
    print(f"An unexpected error occurred during file loading: {e}")
    exit()

# Apply base cleaning to all columns that might contain UKHLS codes
all_cols_to_clean = [col for col in df.columns if col not in ['pidp', 'pid', 'n_hidp']]
df = clean_missing_codes(df, all_cols_to_clean)

Successfully loaded data from: /Users/wanderer/DSS_Thesis/data/processed/n_indresp_cleaned_02.csv


In [49]:
# ==============================================================================
# 1. Feature Engineering: Mental Health Status (Outcome Variable) - USER CODE
# ==============================================================================

# 1.1 Define Primary (Associated) and Secondary (Other) MH Variable Groups
primary_mh_vars = [
    'n_mhcond19', 'n_mhcond2', 'n_mhcond5', 'n_mhcond13', 
    'n_mhcond17', 'n_mhcond18', 'n_mhcond97'
]
secondary_mh_vars_no_none = [ # Exclude n_mhcond96 from the positive check for now
    'n_mhcond8', 'n_mhcond9', 'n_mhcond6', 'n_mhcond10', 
    'n_mhcond4', 'n_mhcond11', 'n_mhcond12', 'n_mhcond14', 
    'n_mhcond3', 'n_mhcond15', 'n_mhcond16'
]
all_positive_mh_vars = primary_mh_vars + secondary_mh_vars_no_none
all_mh_vars_including_96 = all_positive_mh_vars + ['n_mhcond96']

# 1.2 Clean and Prepare MH Data
# Convert UKHLS missing codes to NaN and ensure float type for calculation.
# This cleaning is handled by the initial clean_missing_codes call, but we ensure float type.
df[all_mh_vars_including_96] = df[all_mh_vars_including_96].astype(float)

# 1.3 Create Indicator Variables (1.0 = Yes mentioned, NaN = Missing, 0.0 = Not mentioned)
df['primary_mh_indicator'] = df[primary_mh_vars].max(axis=1, skipna=True)
df['secondary_mh_indicator'] = df[secondary_mh_vars_no_none].max(axis=1, skipna=True)
df['any_mh_problem'] = df[all_positive_mh_vars].max(axis=1, skipna=True)

# 1.4 Construct the Final Categorical Variable 'mental_health_status'
df['mental_health_status'] = np.nan # Default to NaN

# --- Prioritization Logic ---

# 1. Associated MH Issue (any primary condition is 1.0)
df.loc[df['primary_mh_indicator'] == 1.0, 
       'mental_health_status'] = 'Associated MH Issue'

# 2. Other MH Issue (secondary condition is 1.0 AND no primary condition is 1.0)
df.loc[(df['primary_mh_indicator'] == 0.0) & (df['secondary_mh_indicator'] == 1.0), 
       'mental_health_status'] = 'Other MH Issue'

# 3. No Reported MH Issue (Max of all positive MH vars must be 0.0)
mh_answered_but_no_problem = (df['any_mh_problem'] == 0.0)
df.loc[mh_answered_but_no_problem, 
       'mental_health_status'] = 'No Reported MH Issue'

# 4. Missing/Inapplicable 
df['mental_health_status'] = df['mental_health_status'].fillna('Missing/Inapplicable')

# 1.5 Clean up auxiliary columns
df = df.drop(columns=['primary_mh_indicator', 'secondary_mh_indicator', 'any_mh_problem'])
print("\n--- Feature Engineering 1: Mental Health Status Completed ---")
print(df['mental_health_status'].value_counts(dropna=False))


# ==============================================================================
# 2. Feature Engineering: Early Career Unemployment Risk - USER CODE
# ==============================================================================

# 2.1 Clean and Prepare Variables
time_vars = ['n_j1mnth', 'n_j1year', 'n_j1endmnth', 'n_j1endyear', 'n_j1none', 'n_j1still', 'n_jbstat']
df = clean_and_convert_to_nullable_int(df, time_vars)

# 2.2 Define Threshold for "Quick Exit"
QUICK_EXIT_THRESHOLD_MONTHS = 12

# 2.3 Calculate First Job Duration (in months)
df['first_job_duration_months'] = np.where(
    df['n_j1year'].notna() & df['n_j1endyear'].notna() & df['n_j1mnth'].notna() & df['n_j1endmnth'].notna(),
    (df['n_j1endyear'] - df['n_j1year']) * 12 + (df['n_j1endmnth'] - df['n_j1mnth']),
    np.nan
)

# 2.4 Construct the Early Career Risk Variable
df['early_career_unemp_risk'] = np.nan 

# Code 2: Never had a paid job
df.loc[df['n_j1none'] == 2, 'early_career_unemp_risk'] = 'Never Employed'

# Code 1: Still in first job
df.loc[df['n_j1still'] == 1, 'early_career_unemp_risk'] = 'Stable Start: Still in First Job'

# A. Early Failure: Duration < Threshold
df.loc[(df['n_j1still'] == 2) & (df['first_job_duration_months'] < QUICK_EXIT_THRESHOLD_MONTHS),
       'early_career_unemp_risk'] = 'Early Failure: Quick Exit'

# B. Stable Start: Left Long-Term Job: Duration >= Threshold
df.loc[(df['n_j1still'] == 2) & (df['first_job_duration_months'] >= QUICK_EXIT_THRESHOLD_MONTHS),
       'early_career_unemp_risk'] = 'Stable Start: Left Long-Term Job'

# C. Extended Unemployment after Early Failure (Unemployed now, n_jbstat=3)
df.loc[(df['early_career_unemp_risk'] == 'Early Failure: Quick Exit') & (df['n_jbstat'] == 3),
       'early_career_unemp_risk'] = 'Extended Unemp after Quick Exit'

# 2.5 Finalize Missing / Inapplicable
df['early_career_unemp_risk'] = df['early_career_unemp_risk'].fillna('Missing/Inapplicable')
       
print("\n--- Feature Engineering 2: Early Career Unemployment Risk Completed ---")
print(df['early_career_unemp_risk'].value_counts(dropna=False))


# ==============================================================================
# 3. Feature Engineering: Employment Volatility (Employment Stability) - USER CODE
# ==============================================================================

# 3.1 Clean and Prepare Variables
volatility_vars = ['n_nnmpsp_dv', 'n_nmpsp_dv', 'n_nunmpsp_dv']
df = clean_and_convert_to_nullable_int(df, volatility_vars)

# 3.2 Construct Total Spells (Job changes/state changes)
df['total_spells_since_last_int'] = df['n_nnmpsp_dv'] + df['n_nmpsp_dv']

# 3.3 Construct Employment Stability Index
df['employment_stability_level'] = np.nan

# Case 1: Highest Stability - Zero Spells
df.loc[df['total_spells_since_last_int'] == 0, 
       'employment_stability_level'] = 'High Stability: No change reported'
       
# Case 2: Moderate Stability - One Employment Spell, Zero Non-employment Spells
df.loc[(df['n_nmpsp_dv'] == 1) & (df['n_nnmpsp_dv'] == 0), 
       'employment_stability_level'] = 'Moderate Stability: Single Employment Spell'
       
# Case 3: Low Stability - Single Non-Employment Spell 
df.loc[(df['n_nmpsp_dv'] == 0) & (df['n_nnmpsp_dv'] >= 1), 
       'employment_stability_level'] = 'Low Stability: Non-Employment Spells Only'
       
# Case 4: High Volatility - Multiple Spells (Threshold >= 3)
df.loc[df['total_spells_since_last_int'] >= 3, 
       'employment_stability_level'] = 'High Volatility: Frequent Changes'

# 3.4 Finalize Missing/Inapplicable
df['employment_stability_level'] = df['employment_stability_level'].fillna('Missing/Inapplicable')

df['unemployment_spells_count'] = df['n_nunmpsp_dv']
df = df.drop(columns=['total_spells_since_last_int'])

print("\n--- Feature Engineering 3: Employment Stability Completed ---")
print(df['employment_stability_level'].value_counts(dropna=False))


# ==============================================================================
# A. GHQ-12 Score (Using Aggregated Variable n_scghq2_dv)
# ==============================================================================
ghq_caseness_var = 'n_scghq2_dv'

# Clean and rename n_scghq2_dv (already cleaned to float by the initial step)
df['ghq12_continuous_score'] = df[ghq_caseness_var].astype(float) 

print("\n--- Feature Engineering A: GHQ-12 Score (Aggregated) Completed ---")


# ==============================================================================
# B. Education Feature: Ordered Categorical Variable (ISCED-11 based) - REVISED
# Goal: Merge Low and Medium Education into 'Non_High_Education' for robustness.
# ==============================================================================

isced_vars = ['n_isced11_dv', 'n_hiqual_dv']
# Assuming clean_and_convert_to_nullable_int is defined and working
# df = clean_and_convert_to_nullable_int(df, isced_vars) 

isced_var = 'n_isced11_dv'
education_mapping_revised = {
    # Non-High Education: ISCED 0-5 (Codes 9, 2, 3, 4, 5)
    9: 'Non_High_Education',    
    2: 'Non_High_Education',    
    3: 'Non_High_Education',    
    4: 'Non_High_Education', 
    5: 'Non_High_Education', 
    
    # High Education: ISCED 6-8 (Codes 6, 7, 8)
    6: 'High_Education',    
    7: 'High_Education',    
    8: 'High_Education',    
}

# Apply the revised mapping
df['education_level_revised'] = df[isced_var].map(education_mapping_revised).fillna('Missing/Inapplicable')

# Define the new ordered categories
education_categories_revised = ['Non_High_Education', 'High_Education', 'Missing/Inapplicable']
df['education_level_revised'] = pd.Categorical(df['education_level_revised'], 
                                             categories=education_categories_revised, 
                                             ordered=True)

# OPTIONAL: Replace the original column name if you want to use 'education_level' everywhere
# Note: You should update your final column list to use 'education_level_revised' if you keep both.
df['education_level'] = df['education_level_revised']
df = df.drop(columns=['education_level_revised'])


print("--- Feature Engineering B: Education Level (Ordered) REVISED: Non-High vs. High Created ---")

# ==============================================================================
# B.1. Feature Engineering: Vocational Qualification Indicator
# Goal: Create a binary indicator if the individual holds any BTEC qualification.
# ==============================================================================

vocational_vars = ['n_btec1', 'n_btec2', 'n_btec3', 'n_btec4']

# 1. Clean and ensure float type for calculation (already handled by initial cleaning, but re-assert)
df = clean_missing_codes(df, vocational_vars)

# 2. Identify Missingness: Check if all BTEC variables are NaN (Inapplicable/Missing)
df['all_btec_nan'] = df[vocational_vars].isnull().all(axis=1)

# 3. Create the binary indicator 'has_vocational_qual'
# Calculate the maximum value across the 4 BTEC columns. If any is 1.0 (Mentioned), the max is 1.0.
df['has_vocational_qual'] = df[vocational_vars].max(axis=1, skipna=True)

# 4. Handle Missingness: If all BTEC variables were missing, set the new indicator to NaN.
df.loc[df['all_btec_nan'], 'has_vocational_qual'] = np.nan

# 5. Convert to nullable integer
df = clean_and_convert_to_nullable_int(df, ['has_vocational_qual'])

# 6. Clean up auxiliary column
df = df.drop(columns=['all_btec_nan'])

print("\n--- Feature Engineering B.1: Vocational Qualification Indicator Completed ---")
print(df['has_vocational_qual'].value_counts(dropna=False))

# ==============================================================================
# C. Control Variables: Binary/Ordinal/Dummy Encoding - CORRECTED
# ==============================================================================

# Note: We assume core_categorical_vars cleaning (including n_sex, n_health, n_finnow) 
# to Int64 has already been executed successfully.

# C.1. Binary Variables (from codes)

# --- CORRECTED CODE FOR 'female' ---
# 1. Initialize the column with 0 (Male)
df['female'] = 0 
# 2. Use .loc to set the '2' (Female) code to 1
df.loc[df['n_sex'] == 2, 'female'] = 1
# 3. Use .loc to set rows where 'n_sex' is missing (NA) to NaN
df.loc[df['n_sex'].isna(), 'female'] = np.nan
df['female'] = df['female'].astype('Int64') # Convert final result to Nullable Int

# --- CORRECTED CODE FOR 'has_disability' ---
# n_health: 1=Yes, 2=No. Recode to 'has_disability' (1=Yes, 0=No).
# 1. Initialize the column with 0 (No)
df['has_disability'] = 0
# 2. Use .loc to set the '1' (Yes) code to 1
df.loc[df['n_health'] == 1, 'has_disability'] = 1
# 3. Use .loc to set rows where 'n_health' is missing (NA) to NaN
df.loc[df['n_health'].isna(), 'has_disability'] = np.nan
df['has_disability'] = df['has_disability'].astype('Int64')

# --- CORRECTED CODE FOR 'financial_difficulty' ---
# n_finnow: 4.0=Quite difficult, 5.0=Very difficult. Recode to 1=Difficulty, 0=No Difficulty.
# 1. Initialize the column with 0 (No Difficulty)
df['financial_difficulty'] = 0
# 2. Use .loc to set the difficulty codes (4, 5) to 1
df.loc[df['n_finnow'].isin([4, 5]), 'financial_difficulty'] = 1
# 3. Use .loc to set rows where 'n_finnow' is missing (NA) to NaN
df.loc[df['n_finnow'].isna(), 'financial_difficulty'] = np.nan
df['financial_difficulty'] = df['financial_difficulty'].astype('Int64')


# C.2. Categorical Variables: One-Hot Encoding (OHE)

# Marital Status (n_marstat)
df['n_marstat'] = df['n_marstat'].astype('category')
marstat_dummies = pd.get_dummies(df['n_marstat'], prefix='marstat', dummy_na=True, drop_first=False)
df = pd.concat([df, marstat_dummies], axis=1)

# Ethnicity (n_racel_dv)
df['n_racel_dv'] = df['n_racel_dv'].astype('category')
racel_dummies = pd.get_dummies(df['n_racel_dv'], prefix='race', dummy_na=True, drop_first=False)
df = pd.concat([df, racel_dummies], axis=1)

print("--- Feature Engineering C: Control Variables (Binary/OHE) Completed ---")

# ==============================================================================
# D. Advanced Employment Features: Status and Duration - (FIXED: Replacing np.where)
# ==============================================================================

# D.1. Current Unemployment Status (From n_jbstat)

# 1. Binary Indicator: Currently Unemployed
df['is_currently_unemployed'] = 0
# Use .loc for assignment based on Int64 column
df.loc[df['n_jbstat'] == 3, 'is_currently_unemployed'] = 1
# Preserve missingness
df.loc[df['n_jbstat'].isna(), 'is_currently_unemployed'] = np.nan
df['is_currently_unemployed'] = df['is_currently_unemployed'].astype('Int64')


# D.2. Key Non-Working/Inactive Indicators (Control Variables)

# D.2.1. Long-Term Sick/Disabled (Value 8.0)
df['inactive_lt_sick'] = 0
df.loc[df['n_jbstat'] == 8, 'inactive_lt_sick'] = 1
df.loc[df['n_jbstat'].isna(), 'inactive_lt_sick'] = np.nan
df['inactive_lt_sick'] = df['inactive_lt_sick'].astype('Int64')

# D.2.2. Looking After Family/Home (Value 6.0)
df['inactive_home_family'] = 0
df.loc[df['n_jbstat'] == 6, 'inactive_home_family'] = 1
df.loc[df['n_jbstat'].isna(), 'inactive_home_family'] = np.nan
df['inactive_home_family'] = df['inactive_home_family'].astype('Int64')


# D.3. Unemployment Duration (Months)
# Note: Duration calculation used only arithmetic, which is safe from this error.

date_vars = ['n_jlendm', 'n_jlendy', 'n_ienddatm', 'n_ienddaty']
# We assume date_vars were cleaned to Int64 earlier
# No change needed for this block, as it handles NA correctly with .notna() and np.where(..., np.nan)
# We re-run it here for completeness:
df['last_unemployment_duration_months'] = np.where(
    df['n_jlendy'].notna() & df['n_ienddaty'].notna() & df['n_jlendm'].notna() & df['n_ienddatm'].notna(),
    (df['n_ienddaty'] - df['n_jlendy']) * 12 + (df['n_ienddatm'] - df['n_jlendm']),
    np.nan
)
df.loc[df['n_jbstat'].isin([1, 2]), 'last_unemployment_duration_months'] = np.nan


# D.4. Job Search Intensity and Motivation (Auxiliary Indicators)

aux_employment_vars = ['n_jbhas', 'n_julk4wk', 'n_julkjb', 'n_jubgn', 'n_jbhad']
# We assume aux_employment_vars were cleaned to Int64 earlier

# D.4.1. Actively Seeking (Core ILO definition for the unemployed)
# n_julk4wk=1 (Yes, looked for work)
df['actively_seeking'] = 0
df.loc[df['n_julk4wk'] == 1, 'actively_seeking'] = 1
df.loc[df['n_julk4wk'].isna(), 'actively_seeking'] = np.nan
df['actively_seeking'] = df['actively_seeking'].astype('Int64')

# D.4.2. Marginal Attachment/Discouraged Worker 
# Condition: Did no paid work last week (n_jbhas=2) AND would like a job (n_julkjb=1)
df['marginally_attached'] = 0
df.loc[(df['n_jbhas'] == 2) & (df['n_julkjb'] == 1), 'marginally_attached'] = 1
# Identify missingness where either component is NA
missing_mask = df['n_jbhas'].isna() | df['n_julkjb'].isna()
df.loc[missing_mask, 'marginally_attached'] = np.nan
df['marginally_attached'] = df['marginally_attached'].astype('Int64')


# D.4.3. Subjective Employment Chance (n_eprosh)
# Recode subjective chance to be predictive (1=likely, 0=unlikely)
# 1/2 = Likely, 3/4 = Unlikely
df['subjective_job_chance_likely'] = 0
df.loc[df['n_eprosh'].isin([1, 2]), 'subjective_job_chance_likely'] = 1
df.loc[df['n_eprosh'].isna(), 'subjective_job_chance_likely'] = np.nan
df['subjective_job_chance_likely'] = df['subjective_job_chance_likely'].astype('Int64')

print("\n--- Feature Engineering D: Advanced Employment Features (Fixed) Completed ---")
print(df[['is_currently_unemployed', 'last_unemployment_duration_months', 'marginally_attached']].describe())

# ==============================================================================
# E. Demographic and Socioeconomic Control Variables (FIXED)
# Goal: Finalize encoding for age, marital status (using n_marstat), ethnicity, and financial perception.
# ==============================================================================

# Variables to ensure initial cleaning/Int64 conversion:
control_vars_to_clean = ['n_age_dv', 'n_health', 'n_marstat', 'n_racel_dv', 
                         'n_urban_dv', 'n_finnow', 'n_finfut']
df = clean_and_convert_to_nullable_int(df, control_vars_to_clean) # Assuming this function is correctly defined

# E.1. Continuous Variable: Age (n_age_dv)
# Age is already cleaned as Int64.

# E.2. Binary Variables (n_urban_dv)
# n_urban_dv: 1=Urban, 2=Rural. Create 'is_urban' (1=Urban, 0=Rural).
df['is_urban'] = 0
df.loc[df['n_urban_dv'] == 1, 'is_urban'] = 1
df.loc[df['n_urban_dv'].isna(), 'is_urban'] = np.nan
df['is_urban'] = df['is_urban'].astype('Int64')

# E.3. Ordinal/Binary Financial Variables 
# n_finnow -> financial_difficulty (already handled in previous steps using .loc)

# n_finfut: Subjective financial situation - future (1=Better, 2=Worse, 3=Same)
df['n_finfut'] = df['n_finfut'].astype('category')
finfut_dummies = pd.get_dummies(df['n_finfut'], prefix='finfut', dummy_na=True, drop_first=False)
df = pd.concat([df, finfut_dummies], axis=1)


# E.4. High-Dimensional Categorical Variable: Ethnicity (n_racel_dv)
# Goal: Group 17 detailed codes into broader, more statistically meaningful categories (White, Asian, Black, etc.).

# Define the grouping map based on standard UK research practice:
ethnic_group_map = {
    1: 'White', 2: 'White', 3: 'White', 4: 'White',
    5: 'Mixed', 6: 'Mixed', 7: 'Mixed', 8: 'Mixed',
    9: 'Asian', 10: 'Asian', 11: 'Asian', 12: 'Asian', 13: 'Asian',
    14: 'Black', 15: 'Black', 16: 'Black',
    17: 'Other', 97: 'Other'
}

df['ethnic_group'] = df['n_racel_dv'].map(ethnic_group_map)

# Apply One-Hot Encoding to the grouped variable
df['ethnic_group'] = df['ethnic_group'].astype('category')
ethnic_dummies = pd.get_dummies(df['ethnic_group'], prefix='race_grp', dummy_na=True, drop_first=False)
df = pd.concat([df, ethnic_dummies], axis=1)


# E.5. Categorical Variable: Marital Status (n_marstat) - FIXED
marital_var_name = 'n_marstat' # Use the correct column name

# 1. Handle the 'Under 16 years' code (0.0) if present, converting it to NaN/NA.
# This ensures that non-adults are excluded from the adult marital status analysis.
df[marital_var_name] = df[marital_var_name].replace({0.0: np.nan})

# 2. Apply One-Hot Encoding
df[marital_var_name] = df[marital_var_name].astype('category')
marstat_dummies_final = pd.get_dummies(df[marital_var_name], prefix='marstat_final', dummy_na=True, drop_first=False)
df = pd.concat([df, marstat_dummies_final], axis=1)


print("\n--- Feature Engineering E: Demographic and Control Variables (Fixed) Completed ---")


print("\n--- Feature Engineering E: Demographic and Control Variables Completed ---")

# ==============================================================================
# F. Feature Engineering: Previous Wave Mental Health Indicators
# ==============================================================================

# F.1. Clean and Prepare Variables (Including previous health and life satisfaction)
prev_mh_vars = ['scghq2_dv_prev', 'sclfsato_prev', 'health_prev', 'sf12mcs_dv_prev']
df = clean_and_convert_to_nullable_float(df, prev_mh_vars)

# F.2. Introduce Previous GHQ-12 Score (0-12 Caseness)
df['prev_ghq12_score'] = df['scghq2_dv_prev'].astype(float)

# F.3. Calculate GHQ Score Change (Current - Previous)
# Positive value means worsening mental health, negative means improvement.
df['ghq12_score_change'] = df['ghq12_continuous_score'] - df['prev_ghq12_score']

print("\n--- Feature Engineering F: Previous Wave MH Indicators Completed ---")

# ==============================================================================
# G. Feature Engineering: Loneliness/Social Isolation Index
# ==============================================================================

lonely_vars = ['n_scisolate', 'n_sclackcom', 'n_scleftout', 'n_sclonely']

# G.1. Clean and Prepare Variables (Assuming they are Likert scale 1-4)
df = clean_and_convert_to_nullable_float(df, lonely_vars)

# G.2. Construct Loneliness Score (Higher value = Higher Loneliness)
# Assuming 1=Most Frequent/Highest Degree, 4=Least Frequent/Lowest Degree.
# Recode: 1->4, 2->3, 3->2, 4->1. Use 5 - value.
df['loneliness_score'] = df[lonely_vars].apply(
    lambda row: sum(5 - x for x in row if pd.notna(x)),
    axis=1
)
# Ensure that if all components are NaN, the final score is NaN.
df['all_lonely_nan'] = df[lonely_vars].isnull().all(axis=1)
df.loc[df['all_lonely_nan'], 'loneliness_score'] = np.nan
df = df.drop(columns=['all_lonely_nan'])

print("\n--- Feature Engineering G: Loneliness/Social Isolation Index Completed ---")

# ==============================================================================
# H. Feature Engineering: Health Behavior (Smoking Status)
# ==============================================================================

smoking_vars = ['n_smoker', 'n_ncigs']
df = clean_and_convert_to_nullable_int(df, smoking_vars)

# H.1. Binary Indicator: Current Smoker
# Recode: 1 -> 1 (Smoker), 2/3 -> 0 (Non-Smoker). Preserve NaN.
df['is_smoker'] = 0
df.loc[df['n_smoker'] == 1, 'is_smoker'] = 1
# Set rows where n_smoker is NaN back to NaN
df.loc[df['n_smoker'].isna(), 'is_smoker'] = np.nan
df['is_smoker'] = df['is_smoker'].astype('Int64')


# H.2. Binary Indicator: Heavy Smoker (e.g., >= 20 cigarettes/day)
# Note: n_ncigs is NaN for non-smokers/missing.
df['heavy_smoker'] = 0
df.loc[df['n_ncigs'] >= 20, 'heavy_smoker'] = 1
# Preserve missingness from n_ncigs
df.loc[df['n_ncigs'].isna(), 'heavy_smoker'] = np.nan
df['heavy_smoker'] = df['heavy_smoker'].astype('Int64')

print("\n--- Feature Engineering H: Health Behavior (Smoking Status) Completed ---")

# ==============================================================================
# I. Feature Engineering: Objective Income and Economic Pressure
# ==============================================================================

income_vars = ['n_fimnnet_dv', 'n_fimnsben_dv']
df = clean_and_convert_to_nullable_float(df, income_vars)

# I.1. Total Net Income (Log transformed)
# Use log1p(x) for smooth transformation, handling 0 or small values safely.
df['log_total_income'] = df['n_fimnnet_dv'].apply(lambda x: np.log1p(x) if pd.notna(x) and x >= 0 else np.nan)


# I.2. Benefit Income Ratio (Dependency on Social Welfare)
df['benefit_income_ratio'] = df['n_fimnsben_dv'] / df['n_fimnnet_dv']

# If total income is non-positive, set to NaN to avoid Inf/errors/invalid ratio
df.loc[df['n_fimnnet_dv'] <= 0, 'benefit_income_ratio'] = np.nan
# If benefit income is 0 and total income > 0, the result will be 0 (correct).

print("\n--- Feature Engineering I: Objective Income and Economic Pressure Completed ---")

# ==============================================================================
# J. Feature Engineering: Socioeconomic Class (NS-SEC)
# ==============================================================================

nssec_var = 'n_jbnssec8_dv'
df = clean_and_convert_to_nullable_int(df, [nssec_var])

# J.1. Categorical Variable: NS-SEC 8-class (Grouped/Mapped)
nssec_map = {
    1: 'Higher_managerial',   # Higher managerial/professional occupations
    2: 'Lower_managerial',    # Lower managerial/professional occupations
    3: 'Intermediate',        # Intermediate occupations
    4: 'Small_employers',     # Small employers and own account workers
    5: 'Lower_supervisory',   # Lower supervisory and technical occupations
    6: 'Semi_routine',        # Semi-routine occupations
    7: 'Routine',             # Routine occupations
    8: 'Never_worked',        # Never worked and long-term unemployed
}
df['nssec_class'] = df[nssec_var].map(nssec_map).fillna('Missing/Inapplicable')
df['nssec_class'] = df['nssec_class'].astype('category')

# J.2. Binary Indicator: High Status Job (NS-SEC 1 or 2)
df['high_status_job'] = 0
df.loc[df[nssec_var].isin([1, 2]), 'high_status_job'] = 1
df.loc[df[nssec_var].isna(), 'high_status_job'] = np.nan
df['high_status_job'] = df['high_status_job'].astype('Int64')

# J.3. One-Hot Encode the Categorical NS-SEC
nssec_dummies = pd.get_dummies(df['nssec_class'], prefix='nssec', drop_first=False)
df = pd.concat([df, nssec_dummies], axis=1)
df = df.drop(columns=['nssec_class']) # Drop the intermediate categorical column

print("\n--- Feature Engineering J: Socioeconomic Class (NS-SEC) Completed ---")

# ==============================================================================
# K. Feature Engineering: Job Exit Reason Indicators
# ==============================================================================

job_end_var = 'n_jbterm1'
df = clean_and_convert_to_nullable_int(df, [job_end_var])

# K.1. Binary Indicator: Job End Due to Health/Stress/Family (Placeholder Codes based on UKHLS-like structure)
# Assuming: 4=Illness/Disability, 5=Pregnancy, 6=Stress/Mental Health
health_exit_codes = [4, 5, 6] 
df['job_end_due_to_health'] = 0
df.loc[df[job_end_var].isin(health_exit_codes), 'job_end_due_to_health'] = 1
df.loc[df[job_end_var].isna(), 'job_end_due_to_health'] = np.nan
df['job_end_due_to_health'] = df['job_end_due_to_health'].astype('Int64')

# K.2. Binary Indicator: Involuntary Job End (Placeholder Codes based on UKHLS-like structure)
# Assuming: 1=Dismissed, 2=Made Redundant/Laid Off
involuntary_codes = [1, 2]
df['job_end_involuntary'] = 0
df.loc[df[job_end_var].isin(involuntary_codes), 'job_end_involuntary'] = 1
df.loc[df[job_end_var].isna(), 'job_end_involuntary'] = np.nan
df['job_end_involuntary'] = df['job_end_involuntary'].astype('Int64')

print("\n--- Feature Engineering K: Job Exit Reason Indicators Completed ---")

# ==============================================================================
# L. Feature Engineering: Geographic Region Dummies
# ==============================================================================

region_var = 'n_gor_dv'
df = clean_and_convert_to_nullable_int(df, [region_var])

# L.1. Map GOR codes to Region Names (UK Government Office Regions)
region_map = {1:'NorthEast', 2:'NorthWest', 3:'YorkshireHumber', 4:'EastMidlands', 
              5:'WestMidlands', 6:'EastEngland', 7:'London', 8:'SouthEast',
              9:'SouthWest', 10:'Wales', 11:'Scotland', 12:'N_Ireland'}
df['region_name'] = df[region_var].map(region_map).fillna('Missing/Inapplicable')

# L.2. One-Hot Encode Region
df['region_name'] = df['region_name'].astype('category')
# We use dummy_na=False here because 'Missing/Inapplicable' is an explicit category
region_dummies = pd.get_dummies(df['region_name'], prefix='region', dummy_na=False, drop_first=False) 
df = pd.concat([df, region_dummies], axis=1)
df = df.drop(columns=['region_name'])

print("\n--- Feature Engineering L: Geographic Region Dummies Completed ---")

# ==============================================================================
# 5. Final Data Prep and Save (Complete and Optimized) - UPDATED
# Goal: Keep only final engineered features and essential IDs/controls.
# ==============================================================================

# 1. List of ALL desired final columns
columns_to_keep = [
    # 1. IDs/Raw Continuous/Counters 
    'pidp', 'n_age_dv', 'n_hhsize', 'n_nchild_dv', 'unemployment_spells_count', 
    'n_ncigs', # Raw smoking count (H)
    
    # 2. Outcomes & Previous Wave MH (F)
    'mental_health_status', 'ghq12_continuous_score',
    'prev_ghq12_score', 'ghq12_score_change', 
    
    # 3. Core Predictors (Categorical/Ordinal)
    'early_career_unemp_risk', 'employment_stability_level', 'education_level',
    
    # 4. Employment Features (Binary/Continuous) - Step D, J, K
    'is_currently_unemployed', 'last_unemployment_duration_months', 
    'inactive_lt_sick', 'inactive_home_family',
    'actively_seeking', 'marginally_attached', 'subjective_job_chance_likely',
    'high_status_job', 'job_end_due_to_health', 'job_end_involuntary',
    
    # 5. Controls (Continuous/Ordinal/Simple Binary) - Steps C, E, F, G, H, I
    'female', 'has_disability', 'financial_difficulty', 'is_urban',
    'has_vocational_qual', 'health_prev', 'sclfsato_prev', 'sf12mcs_dv_prev', 
    'loneliness_score', 'is_smoker', 'heavy_smoker',
    'log_total_income', 'benefit_income_ratio',

]

# 6. Add all generated dummy columns (One-Hot Encoded variables)
# Collect all prefixes used for OHE variables across all steps
dummy_prefixes = (\
    'marstat_final_', # Final Marital Status (n_marstat_dv fix)
    'race_grp_',      # Ethnic Grouping (Step E)
    'finfut_',        # Future Financial Status (Step E)
    'nssec_',         # NS-SEC Social Class (Step J)
    'region_',        # Geographic Region (Step L)
    'marstat_',       # Old Marital Status (if any)
    'race_',          # Old Ethnicity (if any)
)

# Dynamically capture all columns starting with the specified prefixes
dummy_cols = [col for col in df.columns if col.startswith(dummy_prefixes)]

# Combine and ensure final columns only include those existing in the DataFrame
final_columns = list(set(columns_to_keep + dummy_cols))
final_columns = [col for col in final_columns if col in df.columns]

# Filter the DataFrame to keep only the final model columns
df_final = df[final_columns].copy()

# Save the optimized file
output_file_name = "n_indresp_FINAL_FEATURES_OPTIMIZED02.csv" # Updated name to reflect new features
output_file_path = os.path.join(processed_dir, output_file_name)
df_final.to_csv(output_file_path, index=False)
print(f"\n--- ALL Feature Engineering Stages Complete. Optimized data saved to: {output_file_path} ---")

/var/folders/rr/h4pmcm_12sd99gy5v8tk3dlc0000gn/T/ipykernel_67833/1751457351.py:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['primary_mh_indicator'] = df[primary_mh_vars].max(axis=1, skipna=True)
/var/folders/rr/h4pmcm_12sd99gy5v8tk3dlc0000gn/T/ipykernel_67833/1751457351.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['secondary_mh_indicator'] = df[secondary_mh_vars_no_none].max(axis=1, skipna=True)
/var/folders/rr/h4pmcm_12sd99gy5v8tk3dlc0000gn/T/ipykernel_67833/1751457351.py:26: PerformanceWarning: DataFrame is 


--- Feature Engineering 1: Mental Health Status Completed ---
mental_health_status
Missing/Inapplicable    26833
No Reported MH Issue     6229
Associated MH Issue      2054
Other MH Issue            355
Name: count, dtype: int64

--- Feature Engineering 2: Early Career Unemployment Risk Completed ---
early_career_unemp_risk
Missing/Inapplicable                29180
Stable Start: Left Long-Term Job     4078
Early Failure: Quick Exit            1171
Stable Start: Still in First Job      691
Never Employed                        275
Extended Unemp after Quick Exit        76
Name: count, dtype: int64

--- Feature Engineering 3: Employment Stability Completed ---
employment_stability_level
Missing/Inapplicable                           14142
Moderate Stability: Single Employment Spell    12422
Low Stability: Non-Employment Spells Only       8269
High Stability: No change reported               606
High Volatility: Frequent Changes                 32
Name: count, dtype: int64

--- Feature E